## 1. Import Necessary Package

In [1]:
!pip install stable-baselines[mpi]==2.10.0
import gym
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import math

from simple_particle import SimpleParticle # Importing Environment
from stable_baselines.common.env_checker import check_env


/home/cmb/anaconda3/envs/microrobot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cmb/anaconda3/envs/microrobot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cmb/anaconda3/envs/microrobot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cmb/anaconda3/envs/microrobot/lib/pyt

In [2]:

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.deepq.policies import FeedForwardPolicy
from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common.noise import AdaptiveParamNoiseSpec, NormalActionNoise
%matplotlib inline

## 2. Instantiating the Environment

In [3]:
# Setting Parameters 
# numParticles, phi, stateBounds, dwellTime, maxSteps
numParticles = 1
phi = 0 

env = SimpleParticle(numParticles = numParticles, 
               phi = phi, 
               stateBounds = [-10.0, 10.0, -5.0, 5.0],
               dwellTime = 1.0, 
               maxSteps = 25)


/home/cmb/anaconda3/envs/microrobot/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## 3. Checking the Environment

In [4]:
# Check the environment
env_test = SimpleParticle(numParticles = numParticles, 
               phi = phi, 
               stateBounds = [-10.0, 10.0, -5.0, 5.0],
               dwellTime = 1.0, 
               maxSteps = 25)

# Check
check_env(env_test, warn=True)

[ 1.69200109 -3.39321911 -0.01937325 -0.89710891]


## 4. Instantiate the Agent

In [5]:
# Custom MLP Policy 
policy_kwargs = dict(net_arch=[256, 256])

# Define the model
model = PPO2('MlpPolicy', env, n_steps = 256, policy_kwargs=policy_kwargs, verbose=1)

Wrapping the env in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




# Callback

In [6]:
best_mean_reward = -np.inf
n_steps = 0

num_episodes = 500

def callback(_locals, _globals):
    """
    Callback is called at each step (timestep) for DQN
    :param_locals: (dict)
    :param_globals: (dict)
    """
    global n_steps, best_mean_reward
    
    if (env.episode) > num_episodes:
        return False
    
    # Returning false will stop training early
    return True

In [7]:
# Create log dir
log_dir = "/tmp/gym/"
os.makedirs(log_dir, exist_ok=True)

# Train the agent
model.learn(total_timesteps=int(1e100), callback=callback)

[-0.10870803  0.73673041  0.3456793   0.08752392]
[-2.5691147   2.08986565 -0.94002356  0.81783611]
[3.9009383  2.58548999 0.56647084 0.94981429]
[-1.36829594 -0.20702352  0.69467463  0.96550307]
[ 1.49762016  1.18007839 -0.45948829  0.37635416]
[-1.63506515  3.56199713  0.53509806  0.80679499]
[ 2.71176191 -3.04542966 -0.67842948  0.14933962]
[ 0.29648745 -3.62740408 -0.90142194  0.57793793]
[ 0.62331251  1.13607756 -0.71115624 -0.20482282]
[-0.90977718 -3.24489785 -0.88838044 -0.30076506]
[ 2.06925079 -0.00621856  0.62387045 -0.35587109]
---------------------------------------
| approxkl           | 2.3833259e-06  |
| clipfrac           | 0.0            |
| explained_variance | -1.1e-05       |
| fps                | 37             |
| n_updates          | 1              |
| policy_entropy     | 1.3862828      |
| policy_loss        | -0.00037735802 |
| serial_timesteps   | 256            |
| time_elapsed       | 2.29e-05       |
| total_timesteps    | 256            |
| value_loss  

[-3.20097577 -2.01524333 -0.13935965  0.34018209]
[ 0.12066976  1.51842438  0.42640219 -0.87808751]
[ 2.7028224  -1.3865415  -0.5570609   0.60810863]
[0.62936555 0.14712184 0.92031749 0.60376005]
[ 2.51906354 -0.20948994 -0.70064396  0.42730604]
[-1.11848787 -3.44173215 -0.26473442  0.1658791 ]
[ 2.89178163 -1.61137354 -0.37921255 -0.12235733]
[ 3.1441759   2.82734827 -0.01930689  0.06638643]
[-0.9702244  -3.11234822  0.33136089  0.89897788]
[-3.05126008  2.26575662  0.20061897 -0.61551629]
[ 3.23351775  2.3698304   0.93008688 -0.16446293]
--------------------------------------
| approxkl           | 4.5224686e-05 |
| clipfrac           | 0.0           |
| explained_variance | 9.88e-05      |
| fps                | 1961          |
| n_updates          | 9             |
| policy_entropy     | 1.3833063     |
| policy_loss        | -0.002145039  |
| serial_timesteps   | 2304          |
| time_elapsed       | 7.66          |
| total_timesteps    | 2304          |
| value_loss         | 29

[-3.52738006  1.45032537 -0.45980853  0.84519512]
[ 3.39462631  0.7168269  -0.09739475  0.56971561]
[-3.50275531e+00  2.24810699e-03 -9.74249963e-01 -4.69305732e-01]
[ 1.58808555  1.12158185 -0.61519858  0.14215642]
[ 3.43985363 -2.63045533  0.35336918  0.87856788]
[ 3.33363282  1.5835495   0.20434819 -0.53995207]
[ 0.19237588 -0.50486953  0.10258311  0.22666742]
[-3.60147452  0.77727009 -0.81994492 -0.96782768]
[ 1.70315869 -3.16054119 -0.58329291  0.38706129]
[ 3.83268727 -1.27752599 -0.25962309  0.37534936]
[ 0.53216078 -2.18318038  0.78284166 -0.39294277]
---------------------------------------
| approxkl           | 4.828572e-06   |
| clipfrac           | 0.0            |
| explained_variance | 0.000229       |
| fps                | 1943           |
| n_updates          | 17             |
| policy_entropy     | 1.3729125      |
| policy_loss        | -0.00011114695 |
| serial_timesteps   | 4352           |
| time_elapsed       | 8.71           |
| total_timesteps    | 4352       

[-5.29896735e-01  3.68488851e+00 -2.04490291e-03 -9.97790977e-01]
[0.76398757 1.48180843 0.93900477 0.90509718]
[-1.42749023  3.19566571 -0.39081647 -0.66583915]
[0.18017469 1.58457534 0.96158309 0.22407252]
[ 0.41734995  0.27517018 -0.35951235 -0.75771405]
[ 2.29933174 -0.53309104  0.99106115 -0.00683625]
[-3.93954038  3.7850343   0.9050957  -0.32858188]
[ 3.4376013  -0.36014301  0.75591642 -0.05839148]
[-1.56450084 -2.78028359 -0.99569078  0.85685514]
[0.05952559 2.63268994 0.52657791 0.51977833]
[ 2.89561219 -1.96454075 -0.26005136  0.90422802]
--------------------------------------
| approxkl           | 0.00011274058 |
| clipfrac           | 0.0           |
| explained_variance | -6.23e-05     |
| fps                | 1909          |
| n_updates          | 25            |
| policy_entropy     | 1.3500617     |
| policy_loss        | 0.0014904502  |
| serial_timesteps   | 6400          |
| time_elapsed       | 9.78          |
| total_timesteps    | 6400          |
| value_loss     

[ 1.56658903  0.10855199 -0.13923078 -0.10386415]
[-0.43771412 -1.73228176 -0.00614303 -0.0639462 ]
[-2.37338885  1.74171967  0.64950029  0.52409733]
[ 3.19692708 -2.0332988  -0.99075656 -0.41357006]
[-3.40645815  1.15253532  0.76067788 -0.2635807 ]
[ 3.200043   -3.76209983  0.09970246  0.61537681]
[-2.88176246 -1.734571    0.97243474 -0.66976548]
[-3.13952189 -0.75097831  0.63777344 -0.53441292]
[ 3.48884543 -0.1646621   0.92261292  0.75545099]
[ 0.0243146  -2.01844186  0.46151532 -0.90073147]
--------------------------------------
| approxkl           | 0.0007673404  |
| clipfrac           | 0.0           |
| explained_variance | -5.95e-05     |
| fps                | 1851          |
| n_updates          | 33            |
| policy_entropy     | 1.1915874     |
| policy_loss        | -0.0012703055 |
| serial_timesteps   | 8448          |
| time_elapsed       | 10.9          |
| total_timesteps    | 8448          |
| value_loss         | 2279688.5     |
--------------------------------

...

[-1.01342928 -1.20318292  0.57909952 -0.08283856]
[ 1.23605815 -2.15022462  0.27000172  0.86639777]
[ 3.67569486 -0.30392486 -0.36763988 -0.84507527]
[ 1.33032089 -0.47645539  0.29822609  0.86480066]
[-2.11824256  2.7905606  -0.56866639  0.00772135]
[ 1.88098351  0.01718601 -0.76545108 -0.43345977]
[-1.70149456 -0.05047773 -0.51494468  0.95127113]
[ 0.00600183 -1.59305849 -0.71433679  0.61563073]
-------------------------------------
| approxkl           | 0.016053619  |
| clipfrac           | 0.17773438   |
| explained_variance | -9.54e-07    |
| fps                | 2026         |
| n_updates          | 401          |
| policy_entropy     | 0.88493925   |
| policy_loss        | -0.011232359 |
| serial_timesteps   | 102656       |
| time_elapsed       | 60.8         |
| total_timesteps    | 102656       |
| value_loss         | 19524972.0   |
-------------------------------------
[-3.45269588 -2.99625606  0.51977282  0.96415501]
[ 0.2428022  -1.78425709 -0.13517728  0.74279266]
[-2.76

[-2.30669752 -3.12736081 -0.85691109 -0.76063711]
[ 2.13701229 -1.75837996 -0.88018802  0.8553438 ]
[-3.20348882 -1.43514899 -0.03745786  0.2228966 ]
[ 3.01939742 -3.19258487 -0.79959408  0.13664531]
[-3.98793953 -0.09229358 -0.70906478  0.37574627]
--------------------------------------
| approxkl           | 0.00010845818 |
| clipfrac           | 0.0           |
| explained_variance | 7.15e-07      |
| fps                | 1732          |
| n_updates          | 409           |
| policy_entropy     | 0.8893819     |
| policy_loss        | 0.00023189845 |
| serial_timesteps   | 104704        |
| time_elapsed       | 61.9          |
| total_timesteps    | 104704        |
| value_loss         | 15271204.0    |
--------------------------------------
[-3.3524325  -1.7172712   0.7863355   0.83565071]
[ 1.39679672 -2.77343986  0.15063103  0.39598397]
[-2.09725834  2.33185828 -0.03085456 -0.57421797]
[ 3.83634179  3.2683747  -0.21980375 -0.26985958]
[-3.84629339  3.92392881 -0.5778783  -0.113

[-0.91715536 -1.14993414  0.62358264 -0.96230325]
[-2.8630926  -1.27798682  0.82579966  0.34606277]
[3.97979582 3.21522747 0.74684067 0.38640525]
[-3.82553862 -1.69051457 -0.07592758 -0.82850186]
[-2.6050594   3.20583521 -0.76246948  0.01627504]
[-3.52058348 -1.85611365 -0.67243072  0.65526525]
[-0.56377143 -3.9842857  -0.40188392  0.05770268]
--------------------------------------
| approxkl           | 4.5751774e-05 |
| clipfrac           | 0.0           |
| explained_variance | -4.05e-06     |
| fps                | 1947          |
| n_updates          | 417           |
| policy_entropy     | 0.85020787    |
| policy_loss        | 0.0011498205  |
| serial_timesteps   | 106752        |
| time_elapsed       | 62.9          |
| total_timesteps    | 106752        |
| value_loss         | 9369210.0     |
--------------------------------------
[ 2.66105779 -1.24855357  0.28616452 -0.65156733]
[ 2.48636611 -3.210293    0.43561257 -0.67125166]
[-2.89220855 -0.53409135 -0.08652474  0.8096516

[-0.9856127   1.68876735 -0.38947523 -0.86913308]
[ 2.37743585  0.99126731 -0.67535331 -0.10476512]
[ 1.04489606 -2.45133137  0.11070334 -0.87963486]
[ 0.39670323  3.17672107 -0.84905932  0.0102072 ]
[-3.5125482   1.74988214 -0.65712702 -0.74768263]
[-1.01603098 -3.99026106  0.44184661 -0.52866365]
[ 2.08044892  0.77544044  0.4913875  -0.5458022 ]
[-0.8393783  -1.7237295  -0.62023693 -0.28090598]
[ 1.79682986  2.28844352 -0.96687361 -0.95554723]
---------------------------------------
| approxkl           | 2.032167e-05   |
| clipfrac           | 0.0            |
| explained_variance | 2.21e-06       |
| fps                | 2017           |
| n_updates          | 425            |
| policy_entropy     | 0.8394202      |
| policy_loss        | -5.9447775e-05 |
| serial_timesteps   | 108800         |
| time_elapsed       | 64             |
| total_timesteps    | 108800         |
| value_loss         | 5924849.0      |
---------------------------------------
[ 3.04495281 -1.71050523 -0.03

KeyboardInterrupt: 